# Melt basics

In [1]:
import MagmaPandas as mp

Import some melt data from a csv file

In [2]:
melt_file = "./data/melts.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
melt.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,
PI032-04-01,49.402603,14.539062,6.509044,10.058735,9.757536,3.737374,0.697769,0.132497,2.548581,0.289823,0.628502,1.440208,0.078857,0.145452,0.033958,100
PI032-04-02,48.832207,15.116896,6.508618,9.688642,9.927146,3.564377,0.940308,0.135905,2.652552,0.336327,0.672345,1.330865,0.083879,0.165394,0.044540,100
PI041-02-02,49.124916,16.950138,4.861451,9.183852,10.238358,3.788617,1.077791,0.153258,2.804343,0.561031,0.463920,0.655494,0.047654,0.068020,0.021157,100
PI041-03-01,45.941086,15.753453,6.909844,10.996510,10.778833,3.346468,1.144852,0.112117,3.153303,0.551992,0.797715,0.293341,0.079499,0.086863,0.054123,100
PI041-03-03,45.313339,15.909531,6.831676,11.175276,10.966834,3.417548,1.167550,0.088899,3.262463,0.517891,0.821116,0.311131,0.081713,0.079495,0.055540,100


On top of the regular MagmaFrame methods, Melt has additional methods for calculating

- NBO/T,
- melt densities,
- liquidus temperatures,
- Fe3+/Fe2+ ratios,
- melt FeO and Fe2O3,
- olivine-melt Fe-Mg partitioning coefficients and
- H2O-CO2 saturation pressures.

Let's check the current configuration of MagmaPandas ...

In [3]:
print(mp.configuration)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
Δ buffer....................................1
Melt Fe3+/Fe2+........................borisov
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_15
Volatile solubility model......IaconoMarziano
Volatile species........................mixed


... and calculate liquidus temperatures with a melt-only thermometer at 4 kbar. The configuration is currently set to use equation 15 from Putirka (2008).

In [4]:
P_bar = 4e3

temperatures = melt.temperature(P_bar=P_bar)
temperatures

name
PI032-04-01    1443.385914
PI032-04-02    1445.564297
PI041-02-02    1412.625935
PI041-03-01    1470.801711
PI041-03-03    1469.717871
PI041-05-04    1392.562896
PI041-05-06    1417.312468
PI041-07-01    1458.806150
PI041-07-02    1463.178696
PI052-01-02    1381.292654
Name: T_K, dtype: float64

We can switch to a different thermometer by changing the configuration:

In [5]:
mp.configuration.melt_thermometer = "putirka2008_16"

temperatures_new = melt.temperature(P_bar=P_bar)
temperatures_new

name
PI032-04-01    1441.689112
PI032-04-02    1438.624308
PI041-02-02    1395.139632
PI041-03-01    1442.484997
PI041-03-03    1441.510214
PI041-05-04    1370.343045
PI041-05-06    1399.939176
PI041-07-01    1440.598613
PI041-07-02    1439.236321
PI052-01-02    1368.655861
Name: T_K, dtype: float64

With these temperatures we can calculate melt Fe3+/Fe2+ ratios. The configuration is currently set to fO2 buffered at QFM+1 and the Borisov et al. (2018) Fe3+/Fe2+ model.

In [6]:
Fe3Fe2_QFM1 = melt.Fe3Fe2_QFM(T_K=temperatures, P_bar=P_bar)
Fe3Fe2_QFM1

name
PI032-04-01    0.311069
PI032-04-02    0.308575
PI041-02-02    0.282497
PI041-03-01    0.314827
PI041-03-03    0.321354
PI041-05-04    0.296615
PI041-05-06    0.305791
PI041-07-01    0.317636
PI041-07-02    0.313250
PI052-01-02    0.317078
dtype: float64

Changing the fO2 buffer to lower log offset values reduces the amount of Fe3+

In [7]:
mp.configuration.dQFM = 0

melt.Fe3Fe2_QFM(T_K=temperatures, P_bar=P_bar)

name
PI032-04-01    0.193133
PI032-04-02    0.191585
PI041-02-02    0.175393
PI041-03-01    0.195466
PI041-03-03    0.199518
PI041-05-04    0.184159
PI041-05-06    0.189856
PI041-07-01    0.197210
PI041-07-02    0.194487
PI052-01-02    0.196864
dtype: float64

With our Fe3+/Fe2+ ratios we can calculate melt FeO and Fe2O3 contents

In [8]:
melt.FeO_Fe2O3_calc(Fe3Fe2=Fe3Fe2_QFM1)

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,Fe2O3,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,49.275581,14.501679,6.492308,10.032872,7.423291,3.727764,0.695975,0.132157,2.542028,0.289078,0.626886,1.436505,0.078655,0.145078,0.033870,2.566271,100.0
PI032-04-02,48.705255,15.077596,6.491697,9.663454,7.566503,3.555110,0.937864,0.135552,2.645656,0.335453,0.670597,1.327405,0.083661,0.164964,0.044424,2.594808,100.0
PI041-02-02,49.001869,16.907682,4.849274,9.160849,7.963149,3.779127,1.075092,0.152874,2.797319,0.559626,0.462758,0.653852,0.047534,0.067850,0.021104,2.500042,100.0
PI041-03-01,45.809442,15.708312,6.890044,10.964999,8.174420,3.336879,1.141571,0.111796,3.144267,0.550411,0.795430,0.292501,0.079271,0.086614,0.053968,2.860077,100.0
PI041-03-03,45.179169,15.862423,6.811448,11.142186,8.275122,3.407429,1.164093,0.088636,3.252803,0.516358,0.818685,0.310210,0.081471,0.079259,0.055375,2.955334,100.0
PI041-05-04,47.599008,18.501385,3.861119,9.353541,7.400349,4.577910,1.599178,0.137414,2.475096,0.821372,0.507690,0.459500,0.087350,0.120875,0.058746,2.439466,100.0
PI041-05-06,46.226667,16.926160,4.726061,8.899527,8.871775,3.991369,1.413130,0.167167,3.624544,0.617872,0.638401,0.584214,0.107136,0.127785,0.063211,3.014983,100.0
PI041-07-01,45.718650,15.414290,6.714976,9.793583,8.734319,3.168019,1.287567,0.144541,3.574668,0.561485,0.473303,1.032991,0.075996,0.165371,0.056997,3.083245,100.0
PI041-07-02,45.731116,15.612536,6.652319,10.098012,8.742548,3.197687,1.363625,0.134632,3.543425,0.617623,0.373127,0.617792,0.073761,0.140751,0.057510,3.043535,100.0


Our melts also contain H2O and CO2, so let's calculate volatile saturation pressures. In the configuration we see that we're currently using the Iacono-Marziano et al. (2012) model with mixed CO2-H2O fluids:

In [13]:
print(mp.configuration)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
Δ buffer....................................0
Melt Fe3+/Fe2+........................borisov
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_16
Volatile solubility model.........Allison2022
Volatile species........................mixed


In [14]:
pressures_IM2012 = melt.volatile_saturation_pressure(T_K=temperatures)
pressures_IM2012

|██████████████████████████████| 100% [10/10] in 0.6s (10.19/s) 


name
PI032-04-01    4760.421774
PI032-04-02    5378.750567
PI041-02-02    4897.019731
PI041-03-01    5838.765488
PI041-03-03    6015.052185
PI041-05-04    5257.312523
PI041-05-06    5825.102370
PI041-07-01    4518.033418
PI041-07-02    3923.967613
PI052-01-02    2597.137258
dtype: float64

Lets change the volatile solubility model to Allison et al. (2022):

In [11]:
mp.configuration.volatile_solubility = 'Allison2022'
print(mp.configuration)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
Δ buffer....................................0
Melt Fe3+/Fe2+........................borisov
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_16
Volatile solubility model.........Allison2022
Volatile species........................mixed


In [15]:
pressures_A22 = melt.volatile_saturation_pressure(T_K=temperatures)
pressures_A22

|██████████████████████████████| 100% [10/10] in 0.6s (15.31/s) 


name
PI032-04-01    4760.421774
PI032-04-02    5378.750567
PI041-02-02    4897.019731
PI041-03-01    5838.765488
PI041-03-03    6015.052185
PI041-05-04    5257.312523
PI041-05-06    5825.102370
PI041-07-01    4518.033418
PI041-07-02    3923.967613
PI052-01-02    2597.137258
dtype: float64